## This notebook shows how to use Orchestrator APIs for user experiments

In [ ]:
import os
from fabrictestbed.slice_manager import SliceManager, Status, SliceState
import json

In [ ]:
bastion_public_addr = 'bastion-1.fabric-testbed.net'
bastion_private_ipv4_addr = '192.168.11.226'
bastion_private_ipv6_addr = '2600:2701:5000:a902::c'

bastion_username = '<your bastion username>'

bastion_key_filename = os.environ['HOME'] + "/.ssh/id_rsa_fabric"

ssh_key_file_priv=os.environ['HOME']+"/.ssh/id_rsa"
ssh_key_file_pub=os.environ['HOME']+"/.ssh/id_rsa.pub"

ssh_key_pub = None
with open (ssh_key_file_pub, "r") as myfile:
    ssh_key_pub=myfile.read()
    ssh_key_pub=ssh_key_pub.strip()

In [ ]:
credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
print(f"FABRIC Credential Manager   : {credmgr_host}")

orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"FABRIC Orchestrator         : {orchestrator_host}")

## Create Slice Manager Object
Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters in the refresh call below.

In [ ]:
slice_manager = SliceManager(oc_host=orchestrator_host, 
                             cm_host=credmgr_host ,
                             project_name='all', 
                             scope='all')

# Initialize the slice manager
slice_manager.initialize()

### Orchestrator API example to query for available resources

In [ ]:
status, advertised_topology = slice_manager.resources()

print(f"Status: {status}")
if status == Status.OK:
    print(f"Toplogy: {advertised_topology}")
else:
    print(f"Error: {advertised_topology}")

In [ ]:
if status == Status.OK:
    advertised_topology.draw()

## Create Slice
In Release 1.0, user is expected to create tagged interface and assign the IP addresses manually. Please use the example comands indicated below:

## Configure Slice Parameters



In [ ]:
slice_name = 'MySlice'
site1 = 'MAX'
site2 = 'STAR'
node1_name = 'Node1'
node2_name = 'Node2'
node3_name = 'Node3'
network_service_name='site2site1'
nic1_name = 'node1-nic1'
nic2_name = 'node2-nic1'
nic3_name = 'node3-nic1'
username='centos'
image = 'default_centos_8'
image_type = 'qcow2'
cores = 2
ram = 8
disk = 10

In [ ]:
from fabrictestbed.slice_editor import ExperimentTopology, Capacities, ComponentType, ComponentModelType, ServiceType
# Create topology
t = ExperimentTopology()

# Add node
n1 = t.add_node(name=node1_name, site=site1)

# Set capacities
cap = Capacities()
cap.set_fields(core=cores, ram=ram, disk=disk)

# Set Properties
n1.set_properties(capacities=cap, image_type=image_type, image_ref=image)

# Add node
n2 = t.add_node(name=node2_name, site=site1)

# Set properties
n2.set_properties(capacities=cap, image_type=image_type, image_ref=image)

# Add node
n3 = t.add_node(name=node3_name, site=site2)

# Set properties
n3.set_properties(capacities=cap, image_type=image_type, image_ref=image)


# Shared Cards
n1.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name=nic1_name)
n2.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name=nic2_name)
n3.add_component(model_type=ComponentModelType.SharedNIC_ConnectX_6, name=nic3_name)

# L2STS Service
t.add_network_service(name='sts1', nstype=ServiceType.L2STS,
                      interfaces=[n1.interface_list[0], n2.interface_list[0], n3.interface_list[0]])

# Specify VLAN
for i in t.interface_list:
    if_labels = i.get_property(pname="labels")
    if_labels.vlan = "200"
    i.set_properties(labels=if_labels)

# Generate Slice Graph
slice_graph = t.serialize()

# Request slice from Orchestrator
return_status, slice_reservations = slice_manager.create(slice_name=slice_name, 
                                            slice_graph=slice_graph, 
                                            ssh_key=ssh_key_pub)

if return_status == Status.OK:
    slice_id = slice_reservations[0].get_slice_id()
    print("Submitted slice creation request. Slice ID: {}".format(slice_id))
else:
    print(f"Failure: {slice_reservations}")

## Get the Slice

In [ ]:
import time
def wait_for_slice(slice,timeout=180,interval=10,progress=False):
    timeout_start = time.time()

    if progress: print("Waiting for slice .", end = '')
    while time.time() < timeout_start + timeout:
        return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

        if return_status == Status.OK:
            slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
            if slice.slice_state == "StableOK":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice
            if slice.slice_state == "Closing" or slice.slice_state == "Dead":
                if progress: print(" Slice state: {}".format(slice.slice_state))
                return slice    
        else:
            print(f"Failure: {slices}")
        
        if progress: print(".", end = '')
        time.sleep(interval)
    
    if time.time() >= timeout_start + timeout:
        if progress: print(" Timeout exceeded ({} sec). Slice: {} ({})".format(timeout,slice.slice_name,slice.slice_state))
        return slice    


return_status, slices = slice_manager.slices(excludes=[SliceState.Dead,SliceState.Closing])

if return_status == Status.OK:
    slice = list(filter(lambda x: x.slice_name == slice_name, slices))[0]
    slice = wait_for_slice(slice, progress=True)

print()
print("Slice Name : {}".format(slice.slice_name))
print("ID         : {}".format(slice.slice_id))
print("State      : {}".format(slice.slice_state))
print("Lease End  : {}".format(slice.lease_end))

## Get the Nodes

Retrieve the node information and save the management IP address.


### Get the Topology

In [ ]:
return_status, experiment_topology = slice_manager.get_slice_topology(slice_object=slice)

### Configure Node1

Use ssh to configure eth1 on  node 1.  

```
ip link add link eth1 name eth1.200 type vlan id 200
ip link set dev eth1.200 up
ip addr add 192.168.10.51/24 dev eth1.200
```

In [ ]:
node1 = experiment_topology.nodes[node1_name]

management_ip_node1 = str(node1.get_property(pname='management_ip'))
print("Node Name        : {}".format(node1.name))
print("Management IP    : {}".format(management_ip_node1))
print()


In [ ]:
from ipaddress import ip_address, IPv4Address
def validIPAddress(IP: str) -> str:
    try:
        return "IPv4" if type(ip_address(IP)) is IPv4Address else "IPv6"
    except ValueError:
        return "Invalid"

import paramiko
management_ip = management_ip_node1

key = paramiko.RSAKey.from_private_key_file(ssh_key_file_priv)

bastion=paramiko.SSHClient()
bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
bastion.connect(bastion_public_addr, username=bastion_username, key_filename=bastion_key_filename)


bastion_transport = bastion.get_transport()
if validIPAddress(management_ip) == 'IPv4':
    src_addr = (bastion_private_ipv4_addr, 22)
elif validIPAddress(management_ip) == 'IPv6':
    src_addr = (bastion_private_ipv6_addr, 22)
else:
    print('Management IP Invalid: {}'.format(management_ip))

dest_addr = (management_ip, 22)
bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
stdin, stdout, stderr = client.exec_command('sudo ip link add link eth1 name eth1.200 type vlan id 200')
stdin, stdout, stderr = client.exec_command('sudo ip link set dev eth1.200 up')
stdin, stdout, stderr = client.exec_command('sudo ip addr add 192.168.10.51/24 dev eth1.200')

stdin, stdout, stderr = client.exec_command('ifconfig eth1.200')
print (str(stdout.read(),'utf-8').replace('\\n','\n'))


### Configure Node2

Use ssh to configure eth1 on each Node 2.  

```
ip link add link eth1 name eth1.200 type vlan id 200
ip link set dev eth1.200 up
ip addr add 192.168.10.52/24 dev eth1.200
```

In [ ]:
node2 = experiment_topology.nodes[node2_name]

management_ip_node2 = str(node2.get_property(pname='management_ip'))
print("Node Name        : {}".format(node2.name))
print("Management IP    : {}".format(management_ip_node2))
print()


In [ ]:
from ipaddress import ip_address, IPv4Address
def validIPAddress(IP: str) -> str:
    try:
        return "IPv4" if type(ip_address(IP)) is IPv4Address else "IPv6"
    except ValueError:
        return "Invalid"

import paramiko
management_ip = management_ip_node2

key = paramiko.RSAKey.from_private_key_file(ssh_key_file_priv)

bastion=paramiko.SSHClient()
bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
bastion.connect(bastion_public_addr, username=bastion_username, key_filename=bastion_key_filename)


bastion_transport = bastion.get_transport()
if validIPAddress(management_ip) == 'IPv4':
    src_addr = (bastion_private_ipv4_addr, 22)
elif validIPAddress(management_ip) == 'IPv6':
    src_addr = (bastion_private_ipv6_addr, 22)
else:
    print('Management IP Invalid: {}'.format(management_ip))

dest_addr = (management_ip, 22)
bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
stdin, stdout, stderr = client.exec_command('sudo ip link add link eth1 name eth1.200 type vlan id 200')
stdin, stdout, stderr = client.exec_command('sudo ip link set dev eth1.200 up')
stdin, stdout, stderr = client.exec_command('sudo ip addr add 192.168.10.52/24 dev eth1.200')


stdin, stdout, stderr = client.exec_command('ifconfig eth1.200')
print (str(stdout.read(),'utf-8').replace('\\n','\n'))


### Configure Node3

Use ssh to configure eth1 on  node 3.  

```
ip link add link eth1 name eth1.200 type vlan id 200
ip link set dev eth1.200 up
ip addr add 192.168.10.53/24 dev eth1.200
```

In [ ]:
node3 = experiment_topology.nodes[node3_name]

management_ip_node3 = str(node3.get_property(pname='management_ip'))
print("Node Name        : {}".format(node3.name))
print("Management IP    : {}".format(management_ip_node3))
print()


In [ ]:
from ipaddress import ip_address, IPv4Address
def validIPAddress(IP: str) -> str:
    try:
        return "IPv4" if type(ip_address(IP)) is IPv4Address else "IPv6"
    except ValueError:
        return "Invalid"

import paramiko
management_ip = management_ip_node3

key = paramiko.RSAKey.from_private_key_file(ssh_key_file_priv)

bastion=paramiko.SSHClient()
bastion.set_missing_host_key_policy(paramiko.AutoAddPolicy())
bastion.connect(bastion_public_addr, username=bastion_username, key_filename=bastion_key_filename)


bastion_transport = bastion.get_transport()
if validIPAddress(management_ip) == 'IPv4':
    src_addr = (bastion_private_ipv4_addr, 22)
elif validIPAddress(management_ip) == 'IPv6':
    src_addr = (bastion_private_ipv6_addr, 22)
else:
    print('Management IP Invalid: {}'.format(management_ip))

dest_addr = (management_ip, 22)
bastion_channel = bastion_transport.open_channel("direct-tcpip", dest_addr, src_addr)


client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
client.set_missing_host_key_policy(paramiko.AutoAddPolicy())

stdin, stdout, stderr = client.exec_command('sudo ip link add link eth1 name eth1.200 type vlan id 200')
stdin, stdout, stderr = client.exec_command('sudo ip link set dev eth1.200 up')
stdin, stdout, stderr = client.exec_command('sudo ip addr add 192.168.10.53/24 dev eth1.200')

stdin, stdout, stderr = client.exec_command('ifconfig eth1.200')
print (str(stdout.read(),'utf-8').replace('\\n','\n'))


## Delete Slice

In [1]:
return_status, result = slice_manager.delete(slice_object=slice)

print("Response Status {}".format(return_status))
print("Response received {}".format(result))

NameError: name 'slice_manager' is not defined